In [1]:
import pandas as pd

from companies import earning_calls_id_ticker_map

In [2]:
%load_ext autoreload
%autoreload 2

In [29]:
df_earning_calls_sentiment = pd.read_csv("data/text_results/earning_calls_sentiment.csv")
# df_news_sentiment = pd.read_csv("data/text_results/news_sentiment.csv")
df_reviews_sentiment = pd.read_csv("data/text_results/reviews_sentiment.csv", index_col=0)

In [4]:
df_train_car = pd.read_csv("data/train_data_CAR5.csv")
df_test_car = pd.read_csv("data/test_data_CAR5.csv")
df_train_rev = pd.read_csv("data/train_data_REV.csv")
df_test_rev = pd.read_csv("data/test_data_REV.csv")

## Join Earning Calls

In [5]:
df_earning_calls_sentiment.head()

,company,quarter,year,sentiment_score,confidence,complexity
0,1038351,Q1,2010,-0.518244,0.748589,-2.029088
1,1038351,Q2,2010,-0.236819,0.603098,-2.044513
2,1038351,Q3,2008,0.201910,0.571310,-0.693308
3,1038351,Q4,2009,0.345938,0.664661,-2.092329
4,1038352,Q1,2024,-0.792844,0.885112,0.532956


In [6]:
df_earning_calls_sentiment["year"].unique()

array(['2010', '2008', '2009', '2024', '2023', '2022', '2011', '2012',
       '2013', '2014', '2015', '2016', '2007', '2017', '2018', '2019',
       '2020', '2021', 'Unknown', '2006'], dtype=object)

In [7]:
df_earning_calls_sentiment

,company,quarter,year,sentiment_score,confidence,complexity
0,1038351,Q1,2010,-0.518244,0.748589,-2.029088
1,1038351,Q2,2010,-0.236819,0.603098,-2.044513
2,1038351,Q3,2008,0.201910,0.571310,-0.693308
3,1038351,Q4,2009,0.345938,0.664661,-2.092329
4,1038352,Q1,2024,-0.792844,0.885112,0.532956
...,...,...,...,...,...,...
3863,98045865,Q4,2012,0.875903,0.926368,-0.975581
3864,98045865,Q4,2013,0.634635,0.806643,0.580773
3865,98045865,Q4,2014,-0.194173,0.419272,-0.896915
3866,98045865,Q4,2015,-0.047659,0.511664,0.580773


In [8]:
df_earning_calls_sentiment["tic"] = df_earning_calls_sentiment["company"].map(earning_calls_id_ticker_map)

In [9]:
df_earning_calls_sentiment.isna().sum()

company            0
quarter            0
year               0
sentiment_score    0
confidence         0
complexity         0
tic                0
dtype: int64

In [10]:
df_earning_calls_sentiment["datacqtr"] = df_earning_calls_sentiment["year"].astype(str) + df_earning_calls_sentiment["quarter"]

In [11]:
df_earning_calls_sentiment = df_earning_calls_sentiment[["datacqtr", "tic", "sentiment_score", "confidence", "complexity"]].copy()

In [12]:
df_earning_calls_sentiment.rename(
    columns={
        "datacqtr": "datacqtr",
        "tic": "tic",
        "sentiment_score": "earning_calls_sentiment",
        "confidence": "earning_calls_confidence",
        "complexity": "earning_calls_complexity",
    },
    inplace=True,
)

In [13]:
df_earning_calls_sentiment.sort_values(
    by=["tic", "datacqtr"],
    ascending=[True, True],
    inplace=True,
)

In [14]:
df_earning_calls_sentiment = df_earning_calls_sentiment[df_earning_calls_sentiment["datacqtr"] != "UnknownUnknown"]

In [15]:
df_earning_calls_sentiment

,datacqtr,tic,earning_calls_sentiment,earning_calls_confidence,earning_calls_complexity
416,2007Q3,AFCL,0.827717,0.891771,0.423441
419,2007Q4,AFCL,0.694979,0.839299,-2.374602
412,2008Q1,AFCL,0.717063,0.842497,-1.210036
414,2008Q2,AFCL,0.840870,0.910443,-2.217270
417,2008Q3,AFCL,0.747791,0.862192,-2.451725
...,...,...,...,...,...
2245,2023Q4,ZION,0.722900,0.845726,0.408016
2227,2024Q1,ZION,0.792808,0.883398,-0.943189
2233,2024Q2,ZION,0.774977,0.876247,0.392591
2239,2024Q3,ZION,0.702208,0.839332,-0.881490


In [16]:
df_earning_calls_sentiment = df_earning_calls_sentiment.groupby(by=["tic", "datacqtr"]).mean()

In [17]:
df_earning_calls_sentiment = df_earning_calls_sentiment.reset_index()

In [18]:
df_earning_calls_sentiments = []
for ticker in df_earning_calls_sentiment["tic"].unique():
    df_temp: pd.DataFrame = df_earning_calls_sentiment[df_earning_calls_sentiment["tic"] == ticker].copy()
    df_temp = df_temp.sort_values(by="datacqtr", ascending=True)
    df_temp["datacqtr"] = df_temp["datacqtr"].shift(-1)
    df_temp = df_temp.dropna()
    df_earning_calls_sentiments.append(df_temp)
df_earning_calls_sentiment_lagged = pd.concat(df_earning_calls_sentiments)

In [19]:
df_earning_calls_sentiment_lagged

,tic,datacqtr,earning_calls_sentiment,earning_calls_confidence,earning_calls_complexity
0,AFCL,2007Q4,0.827717,0.891771,0.423441
1,AFCL,2008Q1,0.694979,0.839299,-2.374602
2,AFCL,2008Q2,0.717063,0.842497,-1.210036
3,AFCL,2008Q3,0.840870,0.910443,-2.217270
4,AFCL,2008Q4,0.747791,0.862192,-2.451725
...,...,...,...,...,...
3837,ZION,2023Q4,0.702880,0.843617,0.267651
3838,ZION,2024Q1,0.722900,0.845726,0.408016
3839,ZION,2024Q2,0.792808,0.883398,-0.943189
3840,ZION,2024Q3,0.774977,0.876247,0.392591


In [20]:
df_train_car_new = df_train_car.merge(df_earning_calls_sentiment_lagged, on=["tic", "datacqtr"], how="left")
df_train_car_new["earning_calls_sentiment"] = df_train_car_new["earning_calls_sentiment"].fillna(0)
df_train_car_new["earning_calls_confidence"] = df_train_car_new["earning_calls_confidence"].fillna(0.5)
df_train_car_new["earning_calls_complexity"] = df_train_car_new["earning_calls_complexity"].fillna(0)

In [21]:
df_test_car_new = df_test_car.merge(df_earning_calls_sentiment_lagged, on=["tic", "datacqtr"], how="left")
df_test_car_new["earning_calls_sentiment"] = df_test_car_new["earning_calls_sentiment"].fillna(0)
df_test_car_new["earning_calls_confidence"] = df_test_car_new["earning_calls_confidence"].fillna(0.5)
df_test_car_new["earning_calls_complexity"] = df_test_car_new["earning_calls_complexity"].fillna(0)

In [ ]:
df_train_rev_new = df_train_rev.merge(df_earning_calls_sentiment_lagged, on=["tic", "datacqtr"], how="left")
df_train_rev_new["earning_calls_sentiment"] = df_train_rev_new["earning_calls_sentiment"].fillna(0)
df_train_rev_new["earning_calls_confidence"] = df_train_rev_new["earning_calls_confidence"].fillna(0.5)
df_train_rev_new["earning_calls_complexity"] = df_train_rev_new["earning_calls_complexity"].fillna(0)

In [23]:
df_test_rev_new = df_test_rev.merge(df_earning_calls_sentiment_lagged, on=["tic", "datacqtr"], how="left")
df_test_rev_new["earning_calls_sentiment"] = df_test_rev_new["earning_calls_sentiment"].fillna(0)
df_test_rev_new["earning_calls_confidence"] = df_test_rev_new["earning_calls_confidence"].fillna(0.5)
df_test_rev_new["earning_calls_complexity"] = df_test_rev_new["earning_calls_complexity"].fillna(0)

In [24]:
# check length
print(len(df_train_car), len(df_train_car_new))
print(len(df_test_car), len(df_test_car_new))
print(len(df_train_rev), len(df_train_rev_new))
print(len(df_test_rev), len(df_test_rev_new))

8019 8019
1483 1483
7832 7832
1392 1392


## Join News Sentiment

In [25]:
# df_news_sentiment.head()

## Join Reviews Sentiment

In [30]:
df_reviews_sentiment.describe()

,reviews_rating,text_blob_reviews_sentiment,vader_reviews_sentiment_neg,vader_reviews_sentiment_pos,bert_reviews_label,bert_reviews_score
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000
mean,0.606557,0.219700,0.065272,0.248015,0.590013,0.687629
std,0.249097,0.198478,0.057149,0.128750,0.251946,0.098038
min,0.000000,-0.695000,0.000000,0.000000,0.000000,0.256037
25%,0.461320,0.106081,0.031357,0.176570,0.450000,0.654208
50%,0.590971,0.208471,0.059839,0.232642,0.583333,0.690745
75%,0.777083,0.328077,0.087447,0.301083,0.750000,0.733813
max,1.000000,1.000000,0.756000,1.000000,1.000000,0.982982


In [ ]:
df_train_car_new = df_train_car_new.merge(df_reviews_sentiment, on=["tic", "datacqtr"], how="left")
df_train_car_new["reviews_rating"] = df_train_car_new["reviews_rating"].fillna(0.5)
df_train_car_new["text_blob_reviews_sentiment"] = df_train_car_new["text_blob_reviews_sentiment"].fillna(0)
df_train_car_new["vader_reviews_sentiment_neg"] = df_train_car_new["vader_reviews_sentiment_neg"].fillna(0.5)
df_train_car_new["vader_reviews_sentiment_pos"] = df_train_car_new["vader_reviews_sentiment_pos"].fillna(0.5)
df_train_car_new["bert_reviews_label"] = df_train_car_new["bert_reviews_label"].fillna(0.5)
df_train_car_new["bert_reviews_score"] = df_train_car_new["bert_reviews_score"].fillna(0.5)

In [35]:
df_test_car_new = df_test_car_new.merge(df_reviews_sentiment, on=["tic", "datacqtr"], how="left")
df_test_car_new["reviews_rating"] = df_test_car_new["reviews_rating"].fillna(0.5)
df_test_car_new["text_blob_reviews_sentiment"] = df_test_car_new["text_blob_reviews_sentiment"].fillna(0)
df_test_car_new["vader_reviews_sentiment_neg"] = df_test_car_new["vader_reviews_sentiment_neg"].fillna(0.5)
df_test_car_new["vader_reviews_sentiment_pos"] = df_test_car_new["vader_reviews_sentiment_pos"].fillna(0.5)
df_test_car_new["bert_reviews_label"] = df_test_car_new["bert_reviews_label"].fillna(0.5)
df_test_car_new["bert_reviews_score"] = df_test_car_new["bert_reviews_score"].fillna(0.5)

In [36]:
df_train_rev_new = df_train_rev_new.merge(df_reviews_sentiment, on=["tic", "datacqtr"], how="left")
df_train_rev_new["reviews_rating"] = df_train_rev_new["reviews_rating"].fillna(0.5)
df_train_rev_new["text_blob_reviews_sentiment"] = df_train_rev_new["text_blob_reviews_sentiment"].fillna(0)
df_train_rev_new["vader_reviews_sentiment_neg"] = df_train_rev_new["vader_reviews_sentiment_neg"].fillna(0.5)
df_train_rev_new["vader_reviews_sentiment_pos"] = df_train_rev_new["vader_reviews_sentiment_pos"].fillna(0.5)
df_train_rev_new["bert_reviews_label"] = df_train_rev_new["bert_reviews_label"].fillna(0.5)
df_train_rev_new["bert_reviews_score"] = df_train_rev_new["bert_reviews_score"].fillna(0.5)

In [37]:
df_test_rev_new = df_test_rev_new.merge(df_reviews_sentiment, on=["tic", "datacqtr"], how="left")
df_test_rev_new["reviews_rating"] = df_test_rev_new["reviews_rating"].fillna(0.5)
df_test_rev_new["text_blob_reviews_sentiment"] = df_test_rev_new["text_blob_reviews_sentiment"].fillna(0)
df_test_rev_new["vader_reviews_sentiment_neg"] = df_test_rev_new["vader_reviews_sentiment_neg"].fillna(0.5)
df_test_rev_new["vader_reviews_sentiment_pos"] = df_test_rev_new["vader_reviews_sentiment_pos"].fillna(0.5)
df_test_rev_new["bert_reviews_label"] = df_test_rev_new["bert_reviews_label"].fillna(0.5)
df_test_rev_new["bert_reviews_score"] = df_test_rev_new["bert_reviews_score"].fillna(0.5)

In [38]:
# check length
print(len(df_train_car), len(df_train_car_new))
print(len(df_test_car), len(df_test_car_new))
print(len(df_train_rev), len(df_train_rev_new))
print(len(df_test_rev), len(df_test_rev_new))

8019 8019
1483 1483
7832 7832
1392 1392


In [39]:
# countna
print(df_train_car_new.isna().sum().sum())
print(df_test_car_new.isna().sum().sum())
print(df_train_rev_new.isna().sum().sum())
print(df_test_rev_new.isna().sum().sum())

0
0
0
0


In [43]:
df_train_rev_new.sample(frac=1).head(50)

,datacqtr,tic,Total Current Operating Revenue,GDP CHANGE (-1 to 1),UNEMPLOYMENT RATE (0 to 1),PRIME LOAN RATE (0 to 1),DEPOSITS CHANGE (-1 to 1),CONSUMER PRICE INDEX (0 to 1),SAVINGS PER GROSS INCOME (-1 to 1),Net Interest Income,...,Volatility60,earning_calls_sentiment,earning_calls_confidence,earning_calls_complexity,reviews_rating,text_blob_reviews_sentiment,vader_reviews_sentiment_neg,vader_reviews_sentiment_pos,bert_reviews_label,bert_reviews_score
312,2010Q1,ASRV,0.257109,0.530023,0.663121,0.000000,0.042986,0.140501,0.185714,0.148765,...,0.045243,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
2991,2015Q4,FMER,0.521967,0.495268,0.152482,0.008065,0.083757,0.680886,0.871429,0.510389,...,0.008335,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
2020,2019Q2,EBSB,0.396892,0.564247,0.003546,0.450000,0.118656,0.678019,0.871429,0.336407,...,0.003142,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
5687,2007Q3,PNFP,0.354902,0.550664,0.113475,0.985714,0.132454,0.720095,0.614286,0.406149,...,0.022617,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
1067,2011Q4,CARE,0.350805,0.557621,0.535461,0.000000,0.065829,0.485931,0.600000,0.412648,...,0.384940,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
1633,2014Q4,COBZ,0.340015,0.520156,0.223404,0.000000,0.103271,0.518908,0.942857,0.286273,...,0.008785,0.223888,0.603648,0.642471,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
1956,2016Q2,DNBF,0.198227,0.543477,0.141844,0.050000,0.135543,0.747130,0.728571,0.301570,...,0.978189,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
278,2018Q4,AROW,0.329199,0.518220,0.024823,0.405645,0.061901,0.705095,0.871429,0.254823,...,0.012225,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
3298,2014Q3,FRME,0.399249,0.580382,0.262411,0.000000,0.107351,0.525043,0.957143,0.348636,...,0.002489,0.829523,0.903697,1.649705,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000
1342,2013Q4,CFR,0.512961,0.567101,0.354610,0.000000,0.171328,0.474335,0.785714,0.503938,...,0.001215,0.762925,0.873163,1.916552,0.500000,0.000000,0.500000,0.500000,0.500000,0.500000


In [45]:
len(df_train_rev_new.columns) - 2

52

In [46]:
len(df_train_car_new.columns) - 2

35